In [44]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import pandas as pd
import pandas_ta as ta
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns


In [45]:
data = yf.download('nvda', '2014-01-01')


[*********************100%***********************]  1 of 1 completed


In [46]:
class Rsi(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_window = 14
    
    def init(self):
        self.rsi = self.I(ta.rsi, pd.Series(self.data.Close), self.rsi_window)
        
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
            self.sell(tp=0.9*self.data.Close, sl=1.10*self.data.Close)
        elif crossover(self.lower_bound, self.rsi):
            self.position.close()
            self.buy(tp=1.5*self.data.Close, sl=0.9*self.data.Close)
                
            
bt = Backtest(data, Rsi, cash=100000, commission=.002)
bt.run()
bt.plot()

# bt.run()

/Users/niconievadumas/Programacion II/Stock Price Predictor/stockpp/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/niconievadumas/Programacion II/Stock Price Predictor/stockpp/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p17140', ...)

In [47]:
bt.run()['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-23234,29,33,4.303875,4.628250,-7536.528263,-0.075368,2014-02-13,2014-02-20,7 days
1,-19849,43,196,4.658165,4.240000,8300.162006,0.089770,2014-03-06,2014-10-13,221 days
2,23717,196,213,4.248480,5.062500,19306.117779,0.191603,2014-10-13,2014-11-05,23 days
3,-23765,213,229,5.052375,5.247500,-4637.144265,-0.038620,2014-11-05,2014-11-28,23 days
4,-22041,229,281,5.237005,5.562500,-7174.236554,-0.062153,2014-11-28,2015-02-13,77 days
...,...,...,...,...,...,...,...,...,...,...
80,239,2199,2211,125.320140,109.709999,-3730.823606,-0.124562,2022-09-27,2022-10-13,16 days
81,-162,2233,2253,161.855633,185.309998,-3799.607109,-0.144909,2022-11-14,2022-12-13,29 days
82,-114,2282,2287,196.615975,210.000000,-1525.778905,-0.068072,2023-01-26,2023-02-02,7 days
83,-99,2287,2291,209.580000,223.869995,-1414.709517,-0.068184,2023-02-02,2023-02-08,6 days


In [48]:
bt.run()['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2014-01-02,100000.000000,0.000000,NaT
2014-01-03,100000.000000,0.000000,NaT
2014-01-06,100000.000000,0.000000,NaT
2014-01-07,100000.000000,0.000000,NaT
2014-01-08,100000.000000,0.000000,NaT
...,...,...,...
2023-02-03,20786.391819,0.878099,NaT
2023-02-06,20797.281879,0.878035,NaT
2023-02-07,19724.122242,0.884329,NaT
2023-02-08,19631.648233,0.884871,NaT


In [49]:
def optim_func(series):
    return series['Equity Final [$]']/series['Exposure Time [%]']

In [50]:
stats, heatmap = bt.optimize(
    upper_bound = range(50, 90,2),
    lower_bound = range(10, 50,2),
    rsi_window = range(10, 40,2),
    maximize = 'Equity Final [$]',
    return_heatmap = True
)

heatmap.unstack()

/Users/niconievadumas/Programacion II/Stock Price Predictor/stockpp/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 6000 configurations.
  output = _optimize_grid()
/Users/niconievadumas/Programacion II/Stock Price Predictor/stockpp/lib/python3.10/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


rsi_window                         10            12            14  \
upper_bound lower_bound                                             
50          10           4.014960e+03  3.889782e+03  3.895691e+03   
            12           4.014960e+03  3.889782e+03  3.895691e+03   
            14           4.014960e+03  3.889782e+03  3.895691e+03   
            16           4.401747e+03  3.889782e+03  3.895691e+03   
            18           4.300836e+03  4.419415e+03  3.895691e+03   
...                               ...           ...           ...   
88          40           1.846218e+06  2.083751e+06  8.867137e+05   
            42           1.295981e+06  1.205028e+06  6.497161e+05   
            44           1.374217e+06  1.590598e+06  2.523898e+06   
            46           1.529182e+06  1.030849e+06  1.373984e+06   
            48           1.285633e+06  1.440687e+06  1.742536e+06   

rsi_window                         16            18            20  \
upper_bound lower_bound                                             
50          10           6.795116e+03  9.789190e+03  1.141779e+04   
            12           6.795116e+03  9.789190e+03  1.141779e+04   
            14           6.795116e+03  9.789190e+03  1.141779e+04   
            16           6.795116e+03  9.789190e+03  1.141779e+04   
            18           6.795116e+03  9.789190e+03  1.141779e+04   
...                               ...           ...           ...   
88          40           8.613326e+05  8.170345e+05  4.355254e+05   
            42           8.863359e+05  7.102377e+05  8.464411e+05   
            44           8.933607e+05  1.038915e+06  1.203437e+06   
            46           1.312472e+06  5.069553e+05  6.526488e+05   
            48           1.143369e+06  1.311753e+06  6.484348e+05   

rsi_window                         22             24             26  \
upper_bound lower_bound                                               
50          10           8.485274e+03   14074.159112   18833.765025   
            12           8.485274e+03   14074.159112   18833.765025   
            14           8.485274e+03   14074.159112   18833.765025   
            16           8.485274e+03   14074.159112   18833.765025   
            18           8.485274e+03   14074.159112   18833.765025   
...                               ...            ...            ...   
88          40           2.436666e+05  186137.874955   85430.164418   
            42           7.703060e+05  450999.764358  397424.537183   
            44           1.137956e+06  872704.249980  942452.636336   
            46           6.790422e+05  760960.793683  788166.395597   
            48           5.959962e+05  484062.825314  520048.475496   

rsi_window                          28             30             32  \
upper_bound lower_bound                                                
50          10            18982.467763   15031.815604   26213.726395   
            12            18982.467763   15031.815604   26213.726395   
            14            18982.467763   15031.815604   26213.726395   
            16            18982.467763   15031.815604   26213.726395   
            18            18982.467763   15031.815604   26213.726395   
...                                ...            ...            ...   
88          40            93664.220934   91581.028436  123912.947248   
            42           454908.324916  218790.019922  248168.418287   
            44           524549.963881  549253.293732  458987.860635   
            46           719508.490539  735346.999372  465034.061715   
            48           477296.833400  809416.693474  855973.147018   

rsi_window                          34             36             38  
upper_bound lower_bound                                               
50          10            25747.806833   31616.102881   41135.791128  
            12            25747.806833   31616.102881   41135.791128  
            14            25747.806833   31616.102881   411

In [57]:
print(stats['_strategy'].upper_bound)
print(stats['_strategy'].lower_bound)
print(stats['_strategy'].rsi_window)
print(stats['_trades'])

82
48
14
      Size  EntryBar  ExitBar  EntryPrice   ExitPrice           PnL  \
0    25655        16       34    3.897780    4.687500  2.026026e+04   
1   -25706        34       36    4.678125    4.700000 -5.623138e+02   
2   -25518        36       59    4.690600    4.440000  6.394804e+03   
3    28342        59       86    4.448880    4.562500  3.220216e+03   
4    28285        86       89    4.571625    4.522500 -1.389500e+03   
..     ...       ...      ...         ...         ...           ...   
105  19618      2136     2137  169.338000  151.821002 -3.436485e+05   
106  16795      2166     2175  177.333956  169.889999 -1.250212e+05   
107  16762      2175     2179  170.229779  160.199997 -1.681192e+05   
108  16728      2179     2182  160.520397  142.089996 -3.083037e+05   
109  15610      2261     2292  152.263927  230.084999  1.214787e+06   

     ReturnPct  EntryTime   ExitTime Duration  
0     0.202608 2014-01-27 2014-02-21  25 days  
1    -0.004676 2014-02-21 2014-02-25   4 d